In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

In [7]:
''' Performs cross validation procedure 10 times, holding out 20% of training data as the CV set. '''
def cross_validate(x, y):
    mean_err = 0.
    for k in range(10):
        # Split the training data into training and testing sets for cross-validation.
        x_train, x_cv, y_train, y_cv = train_test_split(x, y, test_size=0.2, random_state=k*SEED)

        # Create and fit the Logistic Regression model.
        log_reg = LogisticRegression()
        log_reg.fit(x_train, y_train)

        # Calculate training error.
        preds = log_reg.predict(x_cv)
        #preds = log_reg.predict_proba(x_cv)[:,1]
        fold_err = log_reg.score(x_cv, y_cv)
        mean_err += fold_err
    mean_err /= 10
    return mean_err

In [8]:
train = pd.read_csv('/Users/anthonydepalatis/Documents/Workspace/school/MidtermProject379K/data/train.csv')
test = pd.read_csv('/Users/anthonydepalatis/Documents/Workspace/school/MidtermProject379K/data/test.csv')

